In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sklearn
from sklearn import *
print(sklearn.__version__)
import sys, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

0.20.2


In [3]:
# get current user
user = !id -un
user = user[0]

In [4]:
data_path = f'/Users/{user}/datasets/kaggle/titanic'

In [5]:
!ls {data_path}

gender_submission.csv test.csv
submission.csv        train.csv


In [6]:
train_data_path = f'{data_path}/train.csv'
test_data_path = f'{data_path}/test.csv'
test_submission_path = f'{data_path}/gender_submission.csv'
sumission_path = f'{data_path}/submission.csv'

In [7]:
# column names
df = pd.read_csv(test_data_path)
column_names = list(df)
row_count = len(df)
print("column names: \n",column_names)
print('row count: ', row_count)

column names: 
 ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
row count:  418


In [56]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
    
class CategoricalEncoder(LabelBinarizer):
    def fit(self, X, y=None):
        """this would allow us to fit the model based on the X input."""
        super(CategoricalEncoder, self).fit(X)
    def transform(self, X, y=None):
        return super(CategoricalEncoder, self).transform(X)

    def fit_transform(self, X, y=None):
        return super(CategoricalEncoder, self).fit(X).transform(X)
    
class FirstCharacterExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return [s[0] for s in list(X.flatten())]
    
class ThreeClassesExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        class_mapping = []
        for x in X:
            if(x in ['A', 'B', 'C']):
                class_mapping.append('A')
            if(x in ['D', 'E', 'F']):
                class_mapping.append('B')
            if(x in ['G', 'T', 'Z']):
                class_mapping.append('C')
        return class_mapping

In [57]:
def create_submission(test_data, field_name, predictions=None):
    passengers = test_data[[field_name]]
    if predictions == None:
        # if no predictions provided all predictions are set to zero
        predictions = [0] * row_count
    predictions_series = pd.Series(predictions, index=passengers.index)
    passengers.loc[:, 'Survived'] = predictions_series
    return passengers
    
# submission_df = create_submission(df, "PassengerId")
# submission_df.head()

In [58]:
train_data = pd.read_csv(train_data_path)
test_data =  pd.read_csv(test_data_path)
test_submission =  pd.read_csv(test_submission_path)

In [59]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [17]:
set(train_data['Embarked'])
# ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'] Cabin Sex

{'C', 'Q', 'S', nan}

In [244]:
train_data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [272]:
def imputeMedian(X_numeric):
    from sklearn.impute import SimpleImputer
    imputer = SimpleImputer(strategy='median')
    imputer.fit(X_numeric)
    X_numeric = imputer.transform(X_numeric)
    return X_numeric

In [273]:
def trainDecisionTreeModel(X_numeric, y):
    from sklearn import tree
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(X_numeric, y)
    return clf

In [60]:
def trainRandomForestModel(X_numeric, y):
    from sklearn.ensemble import RandomForestClassifier
    clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
    clf = clf.fit(X_numeric, y)
    return clf

In [61]:
def mergeWithSexAttribute(data, sliced):
    sex_cat = data['Sex']
    sex_cat_1hot = label_encoder.fit_transform(sex_cat)
    merged = np.c_[sliced, sex_cat_1hot]
    return merged

In [62]:
from sklearn.preprocessing import LabelBinarizer

def getCabinLabelBinarizer(data):
    cabin_cat = data['Cabin']
    cabin_cat = cabin_cat.fillna('Z')
    cabin_cat_1char = [s[0] for s in list(cabin_cat)]
    binarizer_encoder = LabelBinarizer()
    binarizer_encoder.fit(cabin_cat_1char)
    return binarizer_encoder

def mergeWithCabinAttribute(data, sliced, binarizer_encoder):
    cabin_cat = data['Cabin']
    cabin_cat = cabin_cat.fillna('Z')
    cabin_cat_1char = [s[0] for s in list(cabin_cat)]
    cabin_cat_1char_1hot = binarizer_encoder.transform(cabin_cat_1char)
    print(cabin_cat_1char_1hot.shape)
#     merged = np.column_stack((sliced, cabin_cat_1char_1hot))
    merged = np.c_[sliced, cabin_cat_1char_1hot]
    return merged

In [63]:
def createSimpleSubmission(clf, test_data, binarizer_encoder):
    X_test_numeric = test_data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'] ]
    
    imputed_X_test_numeric = imputeMedian(X_test_numeric)
    print(imputed_X_test_numeric.shape)
    
    imputed_X_test_numeric = mergeWithSexAttribute(test_data, imputed_X_test_numeric)
    print(imputed_X_test_numeric.shape)
    
    imputed_X_test_numeric = mergeWithCabinAttribute(test_data, imputed_X_test_numeric, binarizer_encoder)
    print(imputed_X_test_numeric.shape)
    
    X_passengerIds = test_data['PassengerId']
    y_pred = clf.predict(imputed_X_test_numeric)
    data =list(zip(list(X_passengerIds),list(y_pred)))
    
    modelName = type(clf).__name__
    import csv
    file_name = f"{modelName}.csv"
    with open(file_name, 'w') as f:
        writer = csv.writer(f , lineterminator='\n')
        writer.writerow(['PassengerId','Survived'])
        for tup in data:
            writer.writerow(tup)

In [64]:
y_train = train_data['Survived']
X_numeric = train_data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']]
X_numeric_train = imputeMedian(X_numeric)

X_numeric_train = mergeWithSexAttribute(train_data, X_numeric_train)

train_data_wo_label = train_data.drop(['Survived'], axis=1)
merged = pd.concat((train_data_wo_label, test_data))
cabin_label_binarizer = getCabinLabelBinarizer(merged)
print(cabin_label_binarizer.classes_)

X_numeric_train = mergeWithCabinAttribute(train_data, X_numeric_train, cabin_label_binarizer)

NameError: name 'imputeMedian' is not defined

In [270]:
clf = trainRandomForestModel(X_numeric_train, y_train)
createSimpleSubmission(clf, test_data, cabin_label_binarizer)

In [110]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [114]:
sex_cat = train_data['Sex']
sex_cat_1hot = label_encoder.fit_transform(sex_cat)
print(sex_cat_1hot[:5])
print(type(sex_cat_1hot))

[1 0 0 0 1]
<class 'numpy.ndarray'>


In [23]:
from sklearn.preprocessing import LabelBinarizer
binarizer_encoder = LabelBinarizer()

In [140]:
cabin_cat = train_data['Cabin']
cabin_cat = cabin_cat.fillna('Z')
cabin_cat_1char = [s[0] for s in list(cabin_cat)]
cabin_cat_1char_1hot = binarizer_encoder.fit_transform(cabin_cat_1char)
cabin_cat_1char_1hot

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [52]:
num_attributes = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
cabin_attribute = ['Cabin']

In [ ]:
num_pipeline = Pipeline([
    ('imputer', )
])

In [55]:


cabin_pipeline = Pipeline([
    ('selector', DataFrameSelector(cabin_attribute)),
    ('imputer', SimpleImputer(strategy='constant', fill_value='Z')),
    ('first_char_selector', FirstCharacterExtractor()),
    ('three_classes_extractor', ThreeClassesExtractor()),
    ('categorical_encoder', CategoricalEncoder())
])

cabin_prepared = cabin_pipeline.fit_transform(train_data)
cabin_prepared

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1]])

In [20]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='constant', fill_value='Z')
cabin_cat = pd.DataFrame(train_data['Cabin'])
cabin_cat_imp = imputer.fit_transform(cabin_cat)

In [25]:
from sklearn.base import BaseEstimator, TransformerMixin

class FirstCharacterExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return [s[0] for s in list(X.flatten())]
    
char_extractor = FirstCharacterExtractor()
cabin_first_char = char_extractor.transform(cabin_cat_imp)

['Z',
 'C',
 'Z',
 'C',
 'Z',
 'Z',
 'E',
 'Z',
 'Z',
 'Z',
 'G',
 'C',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'D',
 'Z',
 'A',
 'Z',
 'Z',
 'Z',
 'C',
 'Z',
 'Z',
 'Z',
 'B',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'D',
 'Z',
 'B',
 'C',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'B',
 'C',
 'Z',
 'Z',
 'Z',
 'F',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'F',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'C',
 'Z',
 'Z',
 'Z',
 'E',
 'Z',
 'Z',
 'Z',
 'A',
 'D',
 'Z',
 'Z',
 'Z',
 'Z',
 'D',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'C',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'B',
 'Z',
 'Z',
 'Z',
 'Z',
 'E',
 'D',
 'Z',
 'Z',
 'Z',
 'F',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'D',
 'C',
 'Z',
 'B',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'F',
 'Z',
 'Z',
 'C',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'Z',
 'E'

In [24]:
binarizer_encoder = LabelBinarizer()
embarked_cat = train_data['Embarked']
embarked_cat.head()
#embarked_cat.value_counts()
embarked_cat_1hot = binarizer_encoder.fit_transform(embarked_cat.astype(str))

In [110]:
# X.loc[:, 'Cabin_lvl'] = pd.Series(predictions, index=passengers.index)
# X['Cabin'].value_counts()


array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])